In [57]:
import nltk
from nltk.corpus import wordnet, stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import string

# copypasted this here because imports don't seem to work
def preprocess_caption(caption: str) -> str:
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words("english"))

    tokens = word_tokenize(caption)
    tagged = pos_tag(tokens)

    tagged = [
        (word, tag)
        for word, tag in tagged
        if not word.lower() in stop_words and word not in string.punctuation and not word.isdigit()
    ]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word, _get_wordnet_sentence_position(tag)) for word, tag in tagged]

    return " ".join(tokens)


def _get_wordnet_sentence_position(treebank_tag):
    if treebank_tag.startswith("J"):
        return wordnet.ADJ
    elif treebank_tag.startswith("V"):
        return wordnet.VERB
    elif treebank_tag.startswith("N"):
        return wordnet.NOUN
    elif treebank_tag.startswith("R"):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [58]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.metrics import roc_auc_score
import os

In [59]:
members = pd.read_csv("memorization/audiogen_medium_memorized_audiogen_medium_train.csv")
train_data = pd.read_csv("data/train.csv")
train_data["caption"] = train_data["caption"].map(lambda s: preprocess_caption(s))

In [60]:
Path("to_zip").mkdir(exist_ok=True)

In [61]:
members[:3]

,sample_idx,score,caption,token_eq_0,token_eq_1,token_eq_5,token_eq_14,token_eq_30,cosine_0,cosine_1,cosine_5,cosine_14,cosine_30
0,624,0.0425,Wind blow tunnel,0.014,0.030,0.056,0.118,0.258,0.859782,0.948705,0.935466,0.948903,0.978211
1,2916,0.1100,Low sound engine idling,0.010,0.066,0.080,0.164,0.310,0.811843,0.919321,0.918198,0.955492,0.974826
2,884,0.0465,Airplane engine sound distance,0.016,0.028,0.052,0.132,0.266,0.924421,0.868767,0.882967,0.943403,0.973576


In [62]:
def move_files_with_captions(data, subdir: str):
    sample_indices = data["sample_idx"]
    captions = data["caption"]

    Path(f"to_zip/{subdir}").mkdir(exist_ok=True)
    for file_id, caption in zip(sample_indices, captions):
        os.system(f"scp plgjproboszcz@athena.cyfronet.pl:/net/tscratch/people/plgjproboszcz/zzsn/generated_samples_train/{file_id}_target.wav ./to_zip/{subdir}/")
        os.system(f"scp plgjproboszcz@athena.cyfronet.pl:/net/tscratch/people/plgjproboszcz/zzsn/generated_samples_train/{file_id}_30.wav ./to_zip/{subdir}/")

        caption = caption.lower().replace(" ", "_")

        os.system(f"mv ./to_zip/{subdir}/{file_id}_target.wav ./to_zip/{subdir}/{caption}_true.wav")
        os.system(f"mv ./to_zip/{subdir}/{file_id}_30.wav ./to_zip/{subdir}/{caption}_pred.wav")

move_files_with_captions(members[:3], "vggish")

In [63]:
sorted_members = members.sort_values(by="score", ascending=False)
move_files_with_captions(sorted_members.iloc[:3], "score")

In [64]:
sorted_members = members.sort_values(by="token_eq_30", ascending=False)
move_files_with_captions(sorted_members.iloc[:3], "token_equality")